In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 模型花园 Keras YOLOv8
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_keras_yolov8.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_keras_yolov8.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_keras_yolov8.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

注意：此笔记本已在以下环境中测试过：

* Python 版本 = 3.9

您可以直接在 Colab 中打开此笔记本，或创建[谷歌托管](https://cloud.google.com/vertex-ai/docs/workbench/managed/create-instance)或[用户托管](https://cloud.google.com/vertex-ai/docs/workbench/user-managed/create-new)的 Workbench 实例。

## 概述

这篇笔记展示了如何在Vertex AI Model Garden中使用[Keras YOLOv8](https://keras.io/api/keras_cv/models/tasks/yolo_v8_detector/)。

### 目标

* 对预训练或定制模型进行本地推理

* 在Google Cloud Vertex AI中部署预训练或定制模型

* 在Google Cloud Vertex AI中微调模型

### 成本

本教程使用谷歌云的收费组件：

* Vertex AI
* 云存储

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[云存储
价格](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

资料集

本教程使用的数据集是[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/open_images_v4)中的沙拉类别 -Salads category-。此数据集不需要任何特征工程。在本教程中使用的数据集版本存储在一个公共云存储桶中。经过训练的模型可以预测图像中五种物品类别 -沙拉、海鲜、番茄、烘焙品或奶酪- 的边界框位置和相应类型。

## 安装

安装以下所需的包以执行此笔记本。

In [ ]:
import sys

if "google.colab" in sys.modules:
    # Configs for Colab notebooks.
    ! pip3 install --upgrade --quiet google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

# Configs for all notebooks.
! pip3 install --quiet keras-cv==0.6.1
! pip3 install --quiet keras-core==0.1.0

## 开始之前

### 设置您的谷歌云项目

**无论您使用的是哪种笔记本环境，都需要按照以下步骤操作。**

1. [选择或创建谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您将获得300美元的免费信用额用于计算/存储成本。

1. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保
Cloud SDK 在此笔记本中的所有命令中使用正确的项目。

**注意**: Jupyter 运行带有 `!` 前缀的行作为shell命令，并会将以 `$` 前缀的Python变量插入这些命令中。

### 设置您的项目、区域和存储桶

**如果您不知道您的项目 ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参考支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)

您可以更改Vertex AI使用的 `REGION` 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的信息。

您可以创建一个存储桶来存储中间产物，如数据集、训练模型等。

In [ ]:
# The project and bucket are for experiments below.
PROJECT_ID = ""  # @param {type:"string"}

! gcloud config set project $PROJECT_ID

# The form for BUCKET_URI is gs://<bucket-name>.
BUCKET_URI = ""  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}

import os

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(STAGING_BUCKET, "keras_yolov8")

### 初始化Python的Vertex AI SDK

为您的项目初始化Python的Vertex AI SDK。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

### 定义常量和常见函数

In [ ]:
import base64
import io
from datetime import datetime
from typing import Dict, List, Union

import keras_cv
import numpy as np
import tensorflow as tf
import yaml
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from keras_cv import visualization
from PIL import Image

TRAIN_MACHINE_TYPE = "n1-highmem-16"
TRAIN_ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"
TRAIN_NUM_GPU = 2
TRAIN_CONTAINER_URI = (
    "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/keras-yolov8-train"
)
TRAINING_JOB_PREFIX = "train_yolov8"

UPLOAD_JOB_PREFIX = "upload_yolov8"
DEPLOY_JOB_PREFIX = "deploy_yolov8"
SERVING_CONTAINER_URI = (
    "us-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-gpu.2-12:latest"
)
SERVING_ACCELERATOR_TYPE = "NVIDIA_TESLA_T4"
SERVING_MACHINE_TYPE = "n1-standard-4"
SERVING_CONTAINER_ARGS = ["--allow_precompilation", "--allow_compression"]

RESOLUTION = 512


def get_job_name_with_datetime(prefix: str):
    """Generates a job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def load_img(path):
    """Reads image from path and return PIL.Image instance."""
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return Image.fromarray(np.uint8(img)).convert("RGB")


def decode_image(image_str_tensor: tf.string) -> tf.float32:
    """Converts and resizes image bytes to image tensor."""
    image = tf.io.decode_image(image_str_tensor, 3, expand_animations=False)
    image = tf.image.resize(image, (RESOLUTION, RESOLUTION))
    return image


def get_label_map(label_map_yaml_filepath):
    """Returns class id to label mapping given a filepath to the label map."""
    with tf.io.gfile.GFile(label_map_yaml_filepath, "rb") as input_file:
        label_map = yaml.safe_load(input_file.read())["label_map"]
    return label_map


def get_prediction_instances(test_filepath, new_width=-1):
    """Generate instance from image path to pass to Vertex AI Endpoint for prediction."""
    if new_width <= 0:
        with tf.io.gfile.GFile(test_filepath, "rb") as input_file:
            encoded_string = base64.b64encode(input_file.read()).decode("utf-8")
    else:
        img = load_img(test_filepath)
        width, height = img.size
        print("original input image size: ", width, " , ", height)
        new_height = int(height * new_width / width)
        new_img = img.resize((new_width, new_height))
        print("resized input image size: ", new_width, " , ", new_height)
        buffered = io.BytesIO()
        new_img.save(buffered, format="JPEG")
        encoded_string = base64.b64encode(buffered.getvalue()).decode("utf-8")

    instances = [
        {
            "encoded_image": {"b64": encoded_string},
        }
    ]
    return instances


def predict_custom_trained_model(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": f"{location}-aiplatform.googleapis.com"}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    return response.predictions, response.deployed_model_id

## 使用预训练模型在本地进行推理

本部分介绍如何使用在PascalVOC 2012目标检测任务上预训练的YOLOv8-M模型在本地运行推理，该任务包括20个类别。

从云存储加载图像并解码为张量。

In [ ]:
test_filepath = ""  # @param {type:"string"}
img_bytes = tf.io.read_file(test_filepath)
image = tf.expand_dims(decode_image(img_bytes), axis=0)

加载在PascalVOC 2012上预训练的模型。

In [ ]:
model = keras_cv.models.YOLOV8Detector.from_preset(
    "yolo_v8_m_pascalvoc",
    bounding_box_format="xywh",
)

然後進行推論並視覺化結果。

In [ ]:
decoded = model.predict(image)

In [ ]:
# Classes in PascalVOC 2012 dataset.
class_ids = [
    "Aeroplane",
    "Bicycle",
    "Bird",
    "Boat",
    "Bottle",
    "Bus",
    "Car",
    "Cat",
    "Chair",
    "Cow",
    "Dining Table",
    "Dog",
    "Horse",
    "Motorbike",
    "Person",
    "Potted Plant",
    "Sheep",
    "Sofa",
    "Train",
    "Tvmonitor",
    "Total",
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

visualization.plot_bounding_box_gallery(
    image,
    value_range=(0, 255),
    rows=1,
    cols=1,
    y_pred=decoded,
    scale=5,
    font_scale=0.7,
    bounding_box_format="xywh",
    class_mapping=class_mapping,
)

## 微调模型
本部分展示了如何使用训练docker微调Keras YOLOv8模型，然后部署到Vertex AI端点资源。接受的数据集格式是CSV格式的，就像[AutoML图像目标检测](https://cloud.google.com/vertex-ai/docs/image-data/object-detection/prepare-data#input-files)的格式一样，不包含`ML_USE`列。

In [ ]:
input_csv_path = "gs://cloud-samples-data/vision/salads.csv"  # @param {type:"string"}

###开始训练工作
以下代码块显示了一些可能设置的超参数。这些设置仅供演示目的。在使用时，`batch_size`、`learning_rate`和`epochs`等参数可能被覆盖。`backbone`必须是以下之一：
* `yolo_v8_xs_backbone`
* `yolo_v8_s_backbone`
* `yolo_v8_m_backbone`
* `yolo_v8_l_backbone`
* `yolo_v8_xl_backbone`
* `yolo_v8_xs_backbone_coco`
* `yolo_v8_s_backbone_coco`
* `yolo_v8_m_backbone_coco`
* `yolo_v8_l_backbone_coco`
* `yolo_v8_xl_backbone_coco`

如果要使用预训练权重的预设模型，请选择`yolo_v8_xs_backbone_coco`、`yolo_v8_s_backbone_coco`、`yolo_v8_m_backbone_coco`、`yolo_v8_l_backbone_coco`、`yolo_v8_xl_backbone_coco`中的一个。

In [ ]:
# Hyperparameters
epochs = 10
learning_rate = 0.0005
fpn_depth = 3
confidence_threshold = 0.02
iou_threshold = 0.3
backbone = "yolo_v8_xl_backbone_coco"

train_job_name = get_job_name_with_datetime(TRAINING_JOB_PREFIX)
model_dir = os.path.join(MODEL_BUCKET, train_job_name)
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAIN_MACHINE_TYPE,
            "accelerator_type": TRAIN_ACCELERATOR_TYPE,
            "accelerator_count": TRAIN_NUM_GPU,
        },
        "replica_count": 1,
        "disk_spec": {
            "boot_disk_type": "pd-ssd",
            "boot_disk_size_gb": 500,
        },
        "container_spec": {
            "image_uri": TRAIN_CONTAINER_URI,
            "command": [],
            "env": [
                {
                    "name": "RESOLUTION",
                    "value": f"{RESOLUTION}",
                },
            ],
            "args": [
                f"--input_csv_path={input_csv_path}",
                f"--output_model_dir={model_dir}",
                f"--epochs={epochs}",
                f"--pretrained_backbone={backbone}",
                f"--fpn_depth={fpn_depth}",
                f"--learning_rate={learning_rate}",
                f"--confidence_threshold={confidence_threshold}",
                f"--iou_threshold={iou_threshold}",
            ],
        },
    }
]

train_job = aiplatform.CustomJob(
    display_name=train_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

train_job.run()

print("The trained model is saved in: ", model_dir)

### 预测
本节展示如何部署模型并进行在线预测。

1. 上传并部署模型
2. 运行预测

In [ ]:
upload_job_name = get_job_name_with_datetime(UPLOAD_JOB_PREFIX)

serving_env = {
    "MODEL_ID": "keras-yolov8",
    "DEPLOY_SOURCE": "notebook",
}

model = aiplatform.Model.upload(
    display_name=upload_job_name,
    artifact_uri=model_dir,
    serving_container_image_uri=SERVING_CONTAINER_URI,
    serving_container_args=SERVING_CONTAINER_ARGS,
    serving_container_environment_variables=serving_env,
)

print("The uploaded model name is: ", upload_job_name)

deploy_model_name = get_job_name_with_datetime(DEPLOY_JOB_PREFIX)

endpoint = model.deploy(
    deployed_model_display_name=deploy_model_name,
    machine_type=SERVING_MACHINE_TYPE,
    traffic_split={"0": 100},
    accelerator_type=SERVING_ACCELERATOR_TYPE,
    accelerator_count=1,
    min_replica_count=1,
    max_replica_count=1,
)
print("The deployed job name is: ", deploy_model_name)

endpoint_id = endpoint.name
print("endpoint id is: ", endpoint_id)

从云存储加载图像，调整大小并编码。

In [ ]:
test_filepath = "gs://cloud-ml-data/img/openimage/1302/4677521502_6f2767039c_o.jpg"  # @param {type:"string"}
image_bytes = tf.io.read_file(test_filepath)
image_resized = tf.expand_dims(decode_image(image_bytes), axis=0)

instances = get_prediction_instances(test_filepath, new_width=640)

predictions, _ = predict_custom_trained_model(
    project=PROJECT_ID, location=REGION, endpoint_id=endpoint_id, instances=instances
)

使用端点运行在线预测并可视化结果。

In [ ]:
predictions_dict = {
    "boxes": tf.expand_dims(predictions[0]["boxes"], axis=0),
    "classes": tf.expand_dims(predictions[0]["classes"], axis=0),
    "confidence": tf.expand_dims(predictions[0]["confidence"], axis=0),
    "num_detections": predictions[0]["num_detections"],
}

label_map = get_label_map(os.path.join(model_dir, "label_map.yaml"))

visualization.plot_bounding_box_gallery(
    image_resized,
    value_range=(0, 255),
    rows=1,
    cols=1,
    y_pred=predictions_dict,
    scale=5,
    font_scale=0.7,
    bounding_box_format="xywh",
    class_mapping=label_map,
)

### 清理

In [ ]:
# Deletes custom train jobs.
train_job.delete()
# Undeploys models and deletes endpoints.
endpoint.delete(force=True)
# Deletes models.
model.delete()

参考资料

- [使用YOLOV8和KerasCV实现高效目标检测](https://keras.io/examples/vision/yolov8/)
- [Keras YOLOv8 API文档](https://keras.io/api/keras_cv/models/tasks/yolo_v8_detector/)
- [Keras YOLOv8主干网络](https://keras.io/api/keras_cv/models/backbones/yolo_v8/)